In [1]:
from numpy import mean, std, dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, ConvLSTM2D
from keras.utils import to_categorical
from keras.optimizers import Adam

In [2]:
# drive path
drive = '/content/drive/My Drive/Colab Notebooks/HAR'

# local path
# local = 'G:/My Drive/Colab Notebooks/HAR'
local = 'G:/HAR'

hapt = '/HAPT Dataset'       # Human Activity Postural Transitions data set
har = '/UCI HAR Dataset'      # Human Activity Recognition data set

path_drive = drive + har
path_local = local + har

In [3]:
using_drive = 1

if using_drive == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  path = path_drive
else:
  path = path_local

Mounted at /content/drive


Gravitational acceleration data files for x, y and z axes: `total_acc_x_train.txt`, `total_acc_y_train.txt` , `total_acc_z_train.txt`.<br>
Body acceleration data files for x, y and z axes: `body_acc_x_train.txt`, `body_acc_y_train.txt`, `body_acc_z_train.txt`.<br>
Body gyroscope data files for x, y and z axes: `body_gyro_x_train.txt`,
`body_gyro_y_train.txt`, `body_gyro_z_train.txt`.<br>
The structure is mirrored in the test directory.

In [4]:
# load a single file as a numpy array
def load_file(filepath):
  dataframe = read_csv(filepath, header=None, delim_whitespace=True, encoding='utf-8', encoding_errors='ignore')
  return dataframe.values

# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)
  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + '/' + group + '/Inertial Signals/'
  # load all 9 files as a single array
  filenames = list()
  # total acceleration
  filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
  # body acceleration
  filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
  # body gyroscope
  filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  # y = load_file(prefix + '/' + group + '/y_' + group + '.txt') # doesn't work with default 'utf-8'
  y = read_csv(prefix + '/' + group + '/y_' + group + '.txt', header=None, encoding='utf-16')
  return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(path=''):
  # print(path)
  # load all train
  trainX, trainy = load_dataset_group('train', path)
  # load all test
  testX, testy = load_dataset_group('test', path)
  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  return trainX, trainy, testX, testy

In [5]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 10, 32
  n_features, n_outputs = trainX.shape[2], trainy.shape[1]
  # reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 4, 32
  trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
  # define hyperparameters
  num_of_filters = 128 # default 64
  filter_size = (1,3) # default (1,3)
  dropout = 0.5 # default 0.5
  dense_layer_nodes = 100 # default 100
  learning_rate = 0.001 # default 0.001 = 1e-3
  # define model
  model = Sequential()
  model.add(ConvLSTM2D(num_of_filters, filter_size, activation='relu', input_shape=(n_steps, 1, n_length, n_features), dropout=dropout))
  model.add(Flatten())
  model.add(Dense(dense_layer_nodes, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(n_outputs, activation='softmax'))
  opt = Adam(learning_rate=learning_rate)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy, model

In [6]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [7]:
# run an experiment
def run_experiment(repeats=10, path=''):
  print("Experiment starting. Loading dataset...")
  # load data
  trainX, trainy, testX, testy = load_dataset(path)
  print("Finished loading dataset, starting model training...")
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score, model = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)
  print(model.summary())

In [ ]:
# run the experiment
run_experiment(path=path)

Experiment starting. Loading dataset...
Finished loading dataset, starting model training...
>#1: 90.159
>#2: 90.329
>#3: 89.650
>#4: 90.567
>#5: 92.501
>#6: 90.499
>#7: 91.551
>#8: 86.970
>#9: 91.619
>#10: 90.567
[90.15948176383972, 90.32914638519287, 89.65049386024475, 90.56667685508728, 92.5008475780487, 90.49881100654602, 91.55073165893555, 86.96979880332947, 91.6185975074768, 90.56667685508728]
Accuracy: 90.441% (+/-1.402)
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_19 (ConvLSTM2D  (None, 1, 30, 128)        210944    
 )                                                               
                                                                 
 flatten_19 (Flatten)        (None, 3840)              0         
                                                                 
 dense_38 (Dense)            (None, 100)               384100    
                 